In [ ]:
import torch 
from torch import nn
from torch.nn import functional as F
import numpy as np
import pandas as pd
from vocab import Vocab
from dataprocessor import DataProcessor
from conv_model import ConvProtein
from utils import *
from tqdm import trange
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
x,y = get_data('./data/train_fixed.csv')

min_v,max_v = min(y),max(y)

trainx,testx,trainy ,testy = train_test_split(x,y,test_size=0.2)

vocab = Vocab()
vocab.from_file()

vectors = load_ptvectors('./vectors.npy')

train_datafeeder = DataProcessor(vocab , trainx,trainy, [min_v,max_v])
test_datafeeder  = DataProcessor(vocab , testx,testy, [min_v,max_v])

In [ ]:
model = ConvProtein(vectors,lr=5e-5,use_cuda=False)
#model = model.to('cpu')

In [ ]:
for i in trange(3001):
    x,y = train_datafeeder.sample(100)
    model.updates(x,y)
    if i% 100 == 0:
        predictions , trues = [] , []
        for testx,testy in test_datafeeder.export(10 , 100):
            preds = model.predicts(testx)
            predictions.extend(preds)
            trues.extend(testy)
        perf = np.corrcoef(predictions,trues)[0,1]
        print('correlation on test dataset is ',perf)
